## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [16]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from openai import AsyncOpenAI
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from sendgrid import SendGridAPIClient


In [ ]:
from dotenv import load_dotenv
from agents import Runner, trace, Agent, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from sendgrid import SendGridAPIClient

In [18]:
load_dotenv(override=True)

True

In [19]:
load_dotenv(override = True)
GEMINi_BASE_URL = 'https://generativelanguage.googleapis.com/v1beta/openai/'
google_api_key = os.getenv("GEMINI_API_KEY")
if google_api_key:
    print("Google api key is present!!")

Google api key is present!!


In [20]:
gemini_client = AsyncOpenAI(base_url = GEMINi_BASE_URL, api_key = google_api_key)

gemini_model = OpenAIChatCompletionsModel(model = 'gemini-2.5-flash' , openai_client = gemini_client)

In [46]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("pranshuwork19@gmail.com")  # Change to your verified sender
    to_email = To("pranshushitthings@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [50]:
instructions1 = "Yor are a sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
    a company that provides a Saas tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write witty, engaging cold emails that are likely to get a response"

instructions3 = "You are a busy sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC@ compliance and preparing for audits, powered by AI. \
    You write consise, to the point cold emails."

In [14]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [51]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions = instructions1,
    model = gemini_model
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions = instructions2,
    model = gemini_model
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions = instructions3,
    model = gemini_model
)

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [27]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush = True)

Subject: Proactive SOC2 Compliance & Audit Readiness for [Company Name]

Dear [Recipient Name],

I'm [Your Name] from ComplAI, and I'm reaching out because I understand that achieving and maintaining SOC2 compliance, especially in preparation for audits, is a significant challenge for companies like [Company Name]. The complexity, manual effort, and time investment often divert critical resources from core innovation and growth.

ComplAI offers an AI-powered SaaS solution specifically designed to streamline your entire SOC2 journey. We transform a traditionally arduous process into an efficient, continuous, and highly accurate one.

Our platform helps you:
*   **Automate evidence collection and control monitoring**, significantly reducing manual burden.
*   **Identify compliance gaps proactively**, allowing for remediation before audits.
*   **Generate audit-ready documentation** with unprecedented speed and precision.
*   **Reduce the time and cost** associated with compliance efforts

In [11]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


Subject: A Smarter Path to SOC2 Compliance & Audit Success

Dear [Prospect Name],

I'm reaching out from ComplAI as we specialize in helping organizations like yours navigate the complexities of SOC2 compliance and prepare for audits with unprecedented efficiency.

We understand that achieving and maintaining SOC2 compliance can be a significant undertaking, often consuming valuable time, resources, and causing considerable stress during audit periods. The manual processes, documentation, and continuous monitoring can distract your team from core business objectives.

At ComplAI, we've developed an AI-powered SaaS tool specifically designed to transform this challenge. Our platform automates much of the compliance process, providing an intelligent, always-on solution for SOC2 readiness. Our AI-driven approach streamlines evidence collection, facilitates continuous monitoring, and provides proactive insights to pinpoint potential gaps. This ensures you're not just compliant, but *audit-

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [1]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]
for output in outputs:
    print(output + "\n\n")

NameError: name 'trace' is not defined

In [52]:
sales_picker = Agent(
    name = "sales_picker",
    instructions = "You pick the best cols sales email fro the given options. \
        Imagine you are a customer and pick the one you are most likely to respond to. \
        Do not give and explanation; repky with the selected email only.",
        model = gemini_model
)

In [17]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [19]:
mnessage = "Write a cold email"

results = await asyncio.gather(
    Runner.run(sales_agent1, message),
    Runner.run(sales_agent2, message),
    Runner.run(sales_agent3, message)
)

outputs = [result.final_output for result in results]

emails = "Cold sales email:\n\n" + "\nEmail:\n\n".join(outputs)

best = await Runner.run(sales_picker, emails)

print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Your SOC2 Audit is Brewing... (Our AI has the coffee, and the answers)

Hi [Prospect Name],

I'm guessing you've either just survived a SOC2 audit, are currently knee-deep in one, or the very thought of it makes your eye twitch like a rogue compliance checklist item.

Let's be honest: SOC2 isn't usually listed as "most exciting part of my job" by anyone not named "Auditor." It's often a tangled web of spreadsheets, frantic evidence gathering, and that nagging feeling you've forgotten something critical. It's less about innovation and more about proving you're not running your company from a beach hut on dial-up (unless it's a *very* secure beach hut).

What if I told you there's a way to turn that administrative dragon into a well-behaved, albeit still slightly paperwork-heavy, puppy?

That's where **ComplAI** waltzes in. We're a SaaS platform, powered by advanced AI, designed to transform your SOC2 journey from a marathon of misery into... well, a brisk, con

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [53]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions = instructions1,
    model = gemini_model
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions = instructions2,
    model = gemini_model
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions = instructions3,
    model = gemini_model
)



In [10]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [24]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='Yor are a sales agent working for ComplAI,     a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.     You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001D17B964410>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [54]:



@function_tool
def send_email(body : str):
    """Send out an email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("pranshuwork19@gmail.com")
    to_email = To("devpramod@yahoo.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body = mail)
    return {"status" : "success"}


In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [26]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001D1797C4C20>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [11]:
tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = "Write a cold sales email")

In [27]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001D178A2F420>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [55]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description= description)
tool2 = sales_agent2.as_tool(tool_name= "sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name = "sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000183BE56B4C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000183BE482C00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

In [56]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000183BE56A5C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000183BE56B560>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

## And now it's time for our Sales Manager - our planning agent

In [57]:
instructions = """ 
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgement of which one is most effective.

3. Use the send_email tool to send the best email (and only the best email) to the user.

Crucial Rules:
 - You must use the sales agent to generate the drafts - do not write them yourself.
 - You must send ONE email using the send_email tool -never more than one.
"""

sales_manager = Agent(name = "Sales Manager", instructions = instructions, tools = tools, model = gemini_model)

message = "Send a cold sales email addressed to the 'Dear CEO'"

result = await Runner.run(sales_manager, message)


In [58]:
print(result
)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    I've reviewed the drafts and selected the second email (from sales_agent2) as the most effective. Its engaging subject line, empathetic tone, and clear articulation of value make it stand out. I've sent this email to your sales prospects.
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [15]:
print(os.getenv("SENDGRID_API_KEY")[:5])

SG.AI


In [19]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model = gemini_model)

message = "Send a cold sales email addressed to 'Dear CEO'"


result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [31]:
subject_instruction = "You can write a subject for a sold sales email. \
    You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body into am HTML email body. \
    You are given a text email body which might have some markdown \
    and you need to convert it into an html email body with simple, clear, compelling layout and design."

subject_writer = Agent(name = "Email subject writer", instructions = subject_instruction, model = gemini_model)
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for cold sales email.")

html_converter = Agent(name = "HTML email body converter", instructions = html_instructions, model = gemini_model)
html_tool = html_converter.as_tool(tool_name = "html_converter", tool_description = "Convert a text email to an HTML email body.")

In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [32]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('pranshuwork19@gmail.com')
    to_email = To('pranshushitthings@gmail.com')
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body = mail)
    return {"status" : "success"}

In [33]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [34]:
instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
    You first use the subject writer to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
    Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name = "Email Manager",
    instructions = instructions,
    tools = tools,
    model = gemini_model,
    handoff_description="Convert an email to HTML and send it"
)

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [35]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001D17B9377E0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001D17B9374C0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [36]:
sales_manager_instructions = """
You are a Sales Manager as CompAI. Your goal is to find the single best cold sales email using the sales_agent tools.
Follow these steps caefully:
1. Generate Drafts: Use all three sales_agent tools generate three different email drafts Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgement of which one is most effective.

3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts - do not write them yourself.
- You must hand off exactly ONE email to the Email Manager - never more than one.
"""

sales_manager = Agent(
    name = "Sales Magaer",
    instructions = sales_manager_instructions,
    tools = tools,
    handoffs = handoffs,
    model = gemini_model
)
message = "Send out a cold sales email addressed to Dear CEO from Pranshu"

result = await Runner.run(sales_manager, message)

In [37]:
print(result)


RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The email has been successfully sent with the subject "Strategic SOC2 Compliance: Minimizing Risk, Maximizing Opportunity" and the formatted HTML body.
- 15 new item(s)
- 6 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [47]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from openai import AsyncOpenAI
import asyncio
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from sendgrid import SendGridAPIClient

In [49]:
load_dotenv(override = True)

GEMINi_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GEMINI_API_KEY")
if google_api_key:
    print("Google api key is present!!")
else:
    print("Error!")

Google api key is present!!


In [50]:
gemini_client = AsyncOpenAI(base_url = GEMINi_BASE_URL, api_key = google_api_key)

gemini_model = OpenAIChatCompletionsModel(model = 'gemini-2.5-flash', openai_client = gemini_client)


In [51]:

@function_tool
def send_job_email(body:str):
    """Sendout an email with the given body to all employers prospects"""
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('pranshuwork19@gmail.com')
    to_email = To('devpramod@yahoo.com')
    content = Content('text/plain', body)
    mail = Mail(from_email, to_email, "Application email", content).get()
    sg.client.mail.send.post(request_body = mail)
    return {"status":"success"}


In [52]:
instructions_01 = """You are a professional email writing assistant skilled in crafting consise, personalized and formal job application, internship oppurtunity or networking emails.
    Write a short, polished email that I can send to a potential employer or hiring manager. The email should:
    - Have a clear, respectful greeting and subject line suggestion.
    - Introduce me briefly (e.g. name, professional background, area of expertise).
    - Express genuine interest in the company or role.
    - Highlight one or two key strengths or achievements relevant to the position.
    - Politely indicate my openness to discuss oppurtunities or share my resume.
    - End with a professional closing and signature format. 
    
    Tone: Formal, confident, consise( 120 - 150 words).
    Avoid unnecessary jargon and repetition

    Assume the following details unless I provide new ones:

    - Name: **Pranshu Devhade**
    - Field: **Artificial Intelligence and Data Science, Machine Learning, Software Development**
    - Experience: **Internship Experince in Machine Learning, AI agents, Next.js, Tailwind, Data-driven systems**
    - Goal: **To connect with potential empolyers or hiring managers for AI or software development roles**
    """

In [53]:
job_agent = Agent(
    name = "Job Applicant",
    instructions = instructions_01,
    model = gemini_model
)
job_agent_tool = job_agent.as_tool(tool_name="job_agent", tool_description="Write a application email to hiring manager.")

In [54]:
job_agent_tool

FunctionTool(name='job_agent', description='Write a application email to hiring manager.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'job_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029BFE449620>, strict_json_schema=True, is_enabled=True)

In [55]:
tools = [job_agent_tool,send_job_email]

In [56]:
job_manager_instructions = """
You are Job Manager Agent. Your goal is to compose and send a professional email to the potential employer or hiring manager using job_agent and 
send_job_email tools.
Follow these steps carefully:
1. Generate Drafts: Use the job_agent to generate the email drafts. Do not proceed until the draft is ready.

2. Evaluate: Review the drafts and decide the whether is professional and best to impress using your judgement. Do not proceed until it is ready.

3. Sending: Send the email using the send_job_email tools

Crucial Rules:
- You must use the job_agent to generate the drafts - do not write them yourself.
- You must send exactly ONE email using the send_job_email tool- never more than one.
"""
job_manager = Agent(
    name = "Job Manager",
    instructions = job_manager_instructions,
    tools = tools,
    model = gemini_model
)
message = "Send out a job application email to Hiring Manager at tech company."

result = await Runner.run(job_manager, message)



In [57]:
print(result)

RunResult:
- Last agent: Agent(name="Job Manager", ...)
- Final output (str):
    I have sent the job application email to the Hiring Manager at the tech company.
- 5 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.